In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

X = pd.read_csv('X_features.csv', index_col=0)
y = pd.read_csv('y_features.csv', index_col=0)
groups = X.pop('Feature_7')

test = pd.read_csv('test_features.csv', index_col=0).drop(columns = 'Feature_7')
sample = pd.read_csv('sample_submission_2.csv')
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test_2.csv').set_index('Id')
weights = df_train.loc[:,'Weight_Intraday':'Weight_Daily']

C:\Users\matze\Documents\Winton\CRISPDM


In [15]:
#Helper
def typecasting(df):
    categorial_cols = ['Feature_20','Feature_5']
    binary_cols = ['Feature_16']

    for col in categorial_cols:
        df[col] = df[col].astype('category')
    
    for col in binary_cols:
        df[col] = (df[col]-1).astype('category')

def trainscore(y_true,y_pred,weights):
    weights = weights.loc[y_true.index,:]
    daily = pd.concat([weights.Weight_Daily]*2,axis=1)
    minute = pd.concat([weights.Weight_Intraday]*60,axis=1)
    weights = pd.concat([minute,daily],axis=1)
    weights.columns = df_train.loc[:,'Ret_121':'Ret_PlusTwo'].columns
    
    
    wmea = (((abs(y_true-y_pred))*weights).values.flatten().sum()/y_true.size)
    
    
    #minute_ret = abs(y_true.loc[:,'Ret_121':'180']-y_pred.loc[:,'Ret_121':'Ret_180'])*weights.Weight_Intraday
    return wmea


def create_core_features(train,test):
    X_train=train.loc[:,'Feature_1':'Ret_MinusOne']
    X_val=test.loc[:,'Feature_1':'Ret_MinusOne']
    from sklearn.preprocessing import Imputer

    typecasting(X_train)
    X_train = pd.get_dummies(X_train,drop_first=True)

    typecasting(X_val)
    X_val = pd.get_dummies(X_val,drop_first=True)

    imputer = Imputer(strategy='mean', axis=0)
    X_train = pd.DataFrame(imputer.fit_transform(X_train), index= X_train.index,columns=X_train.columns)
    X_val = pd.DataFrame(imputer.fit_transform(X_val), index= X_val.index,columns=X_val.columns)
    X_train['Ret_MinutePast'] = train.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    X_val['Ret_MinutePast'] = test.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    return X_train, X_val

### Creating featuresets

In [17]:
core_features, core_features_test = create_core_features(df_train,df_test)
# generated features is X
#Adding the interactions to Featureset
crafted_features = pd.read_csv('BA_Features/X_features_BA.csv', index_col=0)
crafted_features_test = pd.read_csv('BA_Features/test_features_BA.csv', index_col=0)

In [21]:
crafted_features['interaction_3']= crafted_features.PR_120 * crafted_features.grouped_mad_fet_7
crafted_features_test['interaction_3']= crafted_features_test.PR_120 * test.grouped_mad_fet_7

#X['interaction_4']= X.smoothed_minute_mean * X.minute_mad
#test['interaction_4']= test.smoothed_minute_mean * test.minute_mad

crafted_features['interaction_5']= crafted_features.minute_mad * crafted_features.PR_120
crafted_features_test['interaction_5']= crafted_features_test.minute_mad * crafted_features_test.PR_120

crafted_features['interaction_6']= crafted_features.smoothed_minute_mean * crafted_features.grouped_mad_fet_5
crafted_features_test['interaction_6']= crafted_features_test.smoothed_minute_mean * crafted_features_test.grouped_mad_fet_5

crafted_features['interaction_7']= crafted_features.PR_120 * crafted_features.fet_7_Minute_MAD
crafted_features_test['interaction_7']= crafted_features_test.PR_120 * crafted_features_test.fet_7_Minute_MAD

crafted_features['interaction_8']= crafted_features.fet_7_Minute_MAD * crafted_features.fet_7_Minute_Mean
crafted_features_test['interaction_8']= crafted_features_test.fet_7_Minute_MAD * crafted_features_test.fet_7_Minute_Mean

print(crafted_features.shape,crafted_features_test.shape)

(40000, 102) (120000, 102)


In [50]:
all_features = pd.concat([core_features,crafted_features],axis=1).drop('Feature_7',axis='columns')
all_features_test = pd.concat([core_features_test,crafted_features_test],axis=1).drop('Feature_7',axis='columns')

features_and_target = all_features.copy()
features_and_target['Ret_PlusOne'] = y.Ret_PlusOne
features_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
features_and_target = features_and_target


In [105]:
import time
t0=time.time()
corr_spearman = features_and_target.corr(method='spearman').to_csv('BA_Features/spearman_cor.csv')
print("time:", round(time.time()-t0, 3), "s")
corr_pearson = features_and_target.corr(method='pearson').to_csv('BA_Features/pearson_cor.csv')
print("time:", round(time.time()-t0, 3), "s")

time: 109.554 s
time: 112.161 s


In [ ]:
corr_spearman = pd.read_csv('BA_Features/spearman_cor.csv',index_col=0)
corr_pearson = pd.read_csv('BA_Features/pearson_cor.csv',index_col=0)



In [ ]:
abs(corr_pearson.Ret_PlusOne).sort_values(ascending=False)[1:20]

In [ ]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
n=20000

t0 = time.time()
mic_Ret_PlusOne = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
#select10 = SelectKBest(score_func=mutual_info_regression, k=20).fit(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0))
print('Time: ',time.time()-t0)
mic_Ret_PlusTwo = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusTwo.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
mic_Ret_Minute = mutual_info_regression(all_features.sample(n,random_state=0), df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1).sample(n,random_state=0), random_state=0)

print('Time: ',time.time()-t0)
'''

In [132]:
pd.DataFrame([mic_Ret_PlusOne,mic_Ret_PlusTwo,mic_Ret_Minute], index = ['MIC_PlusOne','MIC_PlusTwo','MIC_Minute'],columns=all_features.columns.values).T.to_csv('BA_Features/MIC.csv')

In [133]:
MIC = pd.read_csv('BA_Features/MIC.csv', index_col=0)

## Predicting Ret_PlusOne

### Featuresets

In [ ]:
rfe = RFE(gs.best_estimator_,n_features_to_select=n, step=1, verbose=1)
rfe.fit(train, y.Ret_PlusOne)
print("training time:", round(time.time()-t0, 3), "s")

cols_to_use = train.columns[rfe.get_support(indices=True)]

In [202]:
mic10 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[1:11].index
corr20 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[1:21].index
corr30 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[1:31].index
corrPear = abs(corr_pearson.Ret_PlusOne).sort_values(ascending=False)[1:21].index

In [206]:
featuresets = pd.DataFrame([mic10,mic20,mic30,corr10,corr20,corr30],index=['mic10','mic20','mic30','corr10','corr20','corr30'],columns=list(range(1,31))).T

In [226]:
('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+".csv")

'BA_Submissions/Ridge_mic10.csv'

In [248]:
xxx = pd.DataFrame([name,algo,mae]).T
xxx.append(pd.DataFrame([algo,algo,algo]).T)
[algo,algo,algo]
startrow = writer.book[sheet_name].max_row

['Ridge', 'Ridge', 'Ridge']

### Ridge

In [259]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
train = all_features
test = all_features_test

results = pd.DataFrame(columns=['algo','featureset','fitting_time','zerobenchmark','validationmark','mae','params'])

for name in featuresets.columns:

#name='mic10'
    target= 'Ret_PlusOne'
    cols_to_use = featuresets[name].dropna()
    learner = Ridge()
    param_grid= {'alpha': [20,19,18,17,16,15,12,11,10,8,6,4,2,1,0.5],'normalize':[True],'max_iter':[700]}

    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(all_features[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    #sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+".csv"), index=False)

    ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


'mic10'

'mic20'

'mic30'

'corr10'

'corr20'

'corr30'

In [ ]:
writer = pd.ExcelWriter('BA_Submissions/Results.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [159]:
#rfe
n = 20

t0=time.time()
rfe = RFE(gs.best_estimator_,n_features_to_select=n, step=1, verbose=1)
rfe.fit(train, y.Ret_PlusOne)
print("training time:", round(time.time()-t0, 3), "s")

cols_to_use = train.columns[rfe.get_support(indices=True)]
gs= GridSearchCV(gbm, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(all_features[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )



grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
sample.to_csv('submissions/GBM_featureset_RFE_20.csv', index=False)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

### Huber

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Huber

from sklearn.feature_selection import RFE
#(estimator, n_features_to_select=None, step=1, verbose=0)


train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = Huber()
param_grid= {'alpha': [100,20,19,18,17,16,15,12,11,1,0.1],'epsilon':[1,1.35,1.8],'max_iter':[700]}


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

### Random Forrest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import RandomForestRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = RandomForestRegressor()
param_grid= {'bootstrap': [True, False],#kein plan also auch weg
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],#tiefer
             'max_features': ['auto', 'sqrt'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [1, 2, 4], #höher
             'min_samples_split': [2, 5, 10],#kann wahrscheinlich weg
             'n_estimators': [200, 400, 600]}#mehr bedeuten dauert länger


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import GradientBoostRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = GradientBoostRegressor()
param_grid= {'bootstrap': [True, False],#kein plan also auch weg
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],#tiefer
             'max_features': ['auto', 'sqrt'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [1, 2, 4], #höher
             'min_samples_split': [2, 5, 10],#kann wahrscheinlich weg
             'n_estimators': [200, 400, 600]}#mehr bedeuten dauert länger


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)